In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, losses, models, Model
import tensorflow.keras as K
import cv2
import math
import numpy as np
import os
import glob
import scipy.io as sio
from random import sample

from keras.models import load_model
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support

In [2]:
### ROSE_Youtu
data_path_test_real_RY = '/content/drive/MyDrive/Anti-Spoofing_datasets/ROSE-Youtu_database/test/real/'
data_path_test_fixed_RY = '/content/drive/MyDrive/Anti-Spoofing_datasets/ROSE-Youtu_database/test/attack/'

In [3]:
temp_real = glob.glob(data_path_test_real_RY+'*.mp4')
print(len(temp_real))
temp_attack = glob.glob(data_path_test_fixed_RY+'*.mp4')
print(len(temp_attack))

data_list = temp_real + temp_attack
data_label = np.concatenate((np.zeros(len(temp_real), dtype=int),np.ones(len(temp_attack), dtype=int)), axis=0)

449
1300


In [4]:
def read_all_frames(video_path):
  frame_list = []
  video = cv2.VideoCapture(video_path)
  success = True
  while success:
    success,frame = video.read()
    if success == True:
        frame = cv2.resize(frame, (64, 64), interpolation = cv2.INTER_AREA)  
        frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
        frame_list.append(frame)
    else:
      break
  return np.asarray(frame_list)

In [5]:
def frames_RY(data_path, count):
  c_frames = []
  frames = []
  al_frames = read_all_frames(data_path)
  for i in range(count):
    c_frames.append(al_frames[i])
  frames.append(c_frames)
  return np.asarray(frames)

In [6]:
model_path = '/content/drive/MyDrive/APIN_REVIEW/supervised_contrastive_imagenet/training_RY/sup_con_classifier_custom.h5'
classifier = load_model(model_path)

def prdict_video_score(data_class,video_path,num_frames):
  frames= frames_RY(video_path,num_frames)
  # print(frames.shape)
  frames = np.squeeze(frames)
  # print(frames.shape)

  if(data_class == 0):
    label = np.concatenate((np.ones((frames.shape[0],1), dtype=int),np.zeros((frames.shape[0],1), dtype=int)), axis=1)
    # print(label.shape)
    y_test_pred_RY = classifier.predict(frames)
    y_test_pred_RY = np.round(np.squeeze(y_test_pred_RY))
    # print(y_test_pred_RY.shape)
    predictted_video_score = 1-accuracy_score(label, y_test_pred_RY)
  elif(data_class == 1):
    label = np.concatenate((np.zeros((frames.shape[0],1), dtype=int),np.ones((frames.shape[0],1), dtype=int)), axis=1)
    # print(label.shape)
    y_test_pred_RY = classifier.predict(frames)
    y_test_pred_RY = np.round(np.squeeze(y_test_pred_RY))
    # print(y_test_pred_RY.shape)
    predictted_video_score = accuracy_score(label, y_test_pred_RY)
  else:
    print('YOU DID SOMETHING REALLY WRONG MAN')
  return predictted_video_score

In [7]:
# data_label
# data_list
predicted_label=[]

for j in range(0,len(data_list)):
  predicted_label.append(prdict_video_score(data_class=data_label[j],video_path=data_list[j],num_frames=75))

3/3 [==============================] - 0s 108ms/step


In [8]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)):
        if int(y_actual[i])==int(y_hat[i])==0:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==1:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [9]:
predicted_label = np.asarray(predicted_label)
tp, fp, tn, fn = perf_measure(data_label, predicted_label)
print(tp, fp, tn, fn)

447 2 1241 5


In [10]:
acc_train = (tp+tn)/(tp+tn+fp+fn)

# acc_train = accuracy_score(data_label, predicted_label)
Y_I_train =(tp/(tp+fn)) + (tn/(tn+fp)) - 1
sensitivity_train = tp / (tp + fn)
specificity_train = tn / (tn + fp)
f1score_train = 2 * tp / (2 * tp + fp + fn)
FAR = fp/(fp + tn)
FRR = fn/(fn + tp)
HTER_train = (FAR + FRR)/2
EER = (fp+fn)/(tn+fp+fn+tp)
print('RY Results')
print(70*'-')
print('Acc:', acc_train,'YI:', Y_I_train, 'Sen:', sensitivity_train, 'Spe:', specificity_train, '\n F1:', f1score_train, 'HTER:', HTER_train, 'EER:', EER)

RY Results
----------------------------------------------------------------------
Acc: 0.9958702064896755 YI: 0.9873290426387773 Sen: 0.9889380530973452 Spe: 0.998390989541432 
 F1: 0.9922308546059934 HTER: 0.006335478680611423 EER: 0.004129793510324484
